<a href="https://colab.research.google.com/github/duskvirkus/alias-free-gan/blob/stable/notebooks/GPU_Inference_Alias_Free_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Inference - Alias-Free GAN

by duskvirkus

This is a notebook for generating images and latent space walks with Alias-Free GAN on a Colab GPU instance.

Repository: https://github.com/duskvirkus/alias-free-gan

## GPU check

If this fails change the runtime type in Runtime > Change runtime type > Select GPU.

In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-e2fbc5dc-47d4-f15a-5792-1e21ddf6ad0e)


## Clone Repository

In [ ]:
import os

repo_name = 'alias-free-gan'
git_repo = 'https://github.com/duskvirkus/alias-free-gan.git'
branch_name = 'stable'

working_dir = os.path.join('/content', repo_name)

!git clone --branch {branch_name} {git_repo}
%cd {repo_name}

Cloning into 'alias-free-gan'...
remote: Enumerating objects: 1364, done.
remote: Counting objects: 100% (406/406), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 1364 (delta 227), reused 340 (delta 204), pack-reused 958
Receiving objects: 100% (1364/1364), 112.70 MiB | 23.30 MiB/s, done.
Resolving deltas: 100% (688/688), done.
/content/alias-free-gan


## Connect Drive

Recomended but not nesscary connect to drive to make accessing models and saving outputs easier. If you don't do this you'll have to upload and download from your colab session.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Dependencies

You will may see warnings or errors. Ignore these unless you're having trouble in other cells then include the output if you file an issue.

Issue Link: https://github.com/duskvirkus/alias-free-gan/issues/new

In [ ]:
!python install.py

     |████████████████████████████████| 918 kB 13.4 MB/s 
     |████████████████████████████████| 253 kB 65.7 MB/s 
     |████████████████████████████████| 1.7 MB 57.5 MB/s 
     |████████████████████████████████| 108 kB 65.0 MB/s 
     |████████████████████████████████| 10.1 MB 50.0 MB/s 
     |████████████████████████████████| 114 kB 68.9 MB/s 
     |████████████████████████████████| 37.1 MB 1.2 MB/s 
     |████████████████████████████████| 118 kB 71.6 MB/s 
     |████████████████████████████████| 636 kB 49.9 MB/s 
     |████████████████████████████████| 272 kB 72.6 MB/s 
     |████████████████████████████████| 829 kB 58.2 MB/s 
     |████████████████████████████████| 1.3 MB 46.6 MB/s 
     |████████████████████████████████| 133 kB 77.4 MB/s 
     |████████████████████████████████| 97 kB 7.9 MB/s 
     |████████████████████████████████| 170 kB 68.8 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 294 kB 62.7 MB/s 
     |███████████

### Install imagemagick

Only needed if you want to make gifs.

In [ ]:
!apt install imagemagick

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fo

## Set Model and Save Location

In [ ]:
model_path = 'rosinality-ffhq-800k' # either the path to a model you've trained or the name of a pretrained model
model_size = 256 # 256, 512, or 1024 (you'll need colab high ram instance for 1024 and maybe 512)
out_base_path = '/content/drive/MyDrive/colab-alias-free-generate/ros-ffhq-test-0/'
import os
os.makedirs(out_base_path, exist_ok=True)

## Setup Output Counter

This is will help avoid overwriting some generated files by accident.

In [ ]:
def get_output_count():
  global output_count
  output_count += 1
  return '-' + str(output_count)

output_count = -1

## Generate Single Images

This will generate a folder of images with a seed range. These seeds can be used in the next section for interpolations. Checkout the help for more information on seed values.

In [ ]:
!python scripts/generate_images.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'images' + get_output_count())
!python scripts/generate_images.py \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --seed_stop 9

Using Alias-Free GAN version: 1.1.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Attempting to load pretrained model...
Attempting to download rosinality-ffhq-800k from https://aliasfreegan.sfo3.cdn.digitaloceanspaces.com/rosinality-ffhq-800k.pt
CompletedProcess(args=['wget', '--progress=bar:force', '-O', '/content/alias-free-gan/pretrained/rosinality-ffhq-800k.pt', 'https://aliasfreegan.sfo3.cdn.digitaloceanspaces.com/rosinality-ffhq-800k.pt'], returncode=0, stdout=b'', stderr=b'--2021-0

## Generate Interpolation

Interpolate been seed values.

**Seeds:**

- If less than two seed values are provided enough will be randomly generated.
- Be sure to start and end at the same seed if you want a loop.

A seeds example
```python
  --seeds 10 1 32 10
```

**Easing:**

You may notice there are a bunch of easing options. Linear is default. All the opitons are python ports of example code from [easings.net](https://easings.net/). Follow the link to view the shape of the different easing methods.

**Notes:**

- As always checkout the help for more information on all available options.
- This automatically compensates for the distance between seed to make animation smoother.

Example Output:


In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'interpolate' + get_output_count())
!python scripts/generate_interpolation.py \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48

Using Alias-Free GAN version: 1.1.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Attempting to load pretrained model...
Loading pretrained model from: /content/alias-free-gan/pretrained/rosinality-ffhq-800k.pt


rosinality-ffhq-800k information:
FFHQ 256 model trained by rosinality (source: https://github.com/rosinality/alias-free-gan-pytorch/issues/3). Trained for 800k steps with a batch size of 16.


Generating 48 frames for interpolation with seeds [80733, 88649] with linear easing.
Sa

### Create .mp4 with ffmpeg

In [ ]:
!ffmpeg -framerate 24 -i {out_dir}/frames/frame-%09d.png -c:v libx264 -pix_fmt yuv420p -crf 23 {out_dir}/interpolation.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### Create .gif with imagemagick

In [ ]:
!convert -delay 5 -loop 0 {out_dir}/frames/*.png {out_dir}/interpolation.gif

## Circular Interpolation

Generates interpolation using sin and cos.

**Diameter:**

Diameter value should be between 0.0 and 10.0. If two are passed, the script will treat them as a min and max and generate a random diameter for each vector dimension.

It's recommended you use low values for low frame counts like 1.5 for 36 frames and much higher values for larger frame counts like 6.0 for 480 frames.

**Notes:**

- Seed values are used diffently from generate images and interpolate.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'circular' + get_output_count())
!python scripts/generate_interpolation.py \
  --method circular \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48 \
  --diameter 2.0

Using Alias-Free GAN version: 1.1.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Attempting to load pretrained model...
Loading pretrained model from: /content/alias-free-gan/pretrained/rosinality-ffhq-800k.pt


rosinality-ffhq-800k information:
FFHQ 256 model trained by rosinality (source: https://github.com/rosinality/alias-free-gan-pytorch/issues/3). Trained for 800k steps with a batch size of 16.


Generating 48 frames for circular interpolation with seed 98836.
Saving log of argument

### Create .mp4 with ffmpeg

In [ ]:
!ffmpeg -framerate 24 -i {out_dir}/frames/frame-%09d.png -c:v libx264 -pix_fmt yuv420p -crf 23 {out_dir}/circular.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### Create .gif with imagemagick

In [ ]:
!convert -delay 5 -loop 0 {out_dir}/frames/*.png {out_dir}/circular.gif

## Simplex Noise

Generates a interpolation loop useing open simplex noise.

**Diameter:**

No limits on diameter size. Two diameter values will be treated as min and max.

Same thing as above where smaller diameter values for low frame count and higher for more frames.

**Notes:**

- Seed values are used diffently from generate images and interpolate.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'simplex_noise' + get_output_count())
!python scripts/generate_interpolation.py \
  --method simplex_noise \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48 \
  --diameter 0.3

Using Alias-Free GAN version: 1.1.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Attempting to load pretrained model...
Loading pretrained model from: /content/alias-free-gan/pretrained/rosinality-ffhq-800k.pt


rosinality-ffhq-800k information:
FFHQ 256 model trained by rosinality (source: https://github.com/rosinality/alias-free-gan-pytorch/issues/3). Trained for 800k steps with a batch size of 16.


Generating 48 frames for simplex noise interpolation with seed 71112.
Saving log of arg

### Create .mp4 with ffmpeg

In [ ]:
!ffmpeg -framerate 24 -i {out_dir}/frames/frame-%09d.png -c:v libx264 -pix_fmt yuv420p -crf 23 {out_dir}/noise.mp4

### Create .gif with imagemagick

In [ ]:
!convert -delay 5 -loop 0 {out_dir}/frames/*.png {out_dir}/noise.gif

## Repeat / Load Interpolation

By default each interpolation saves a z_vectors.pt file which can be reloaded to repeat the interpolation. As long as the models have the same style dimensions you can use these across different models.

You can also use another method to generate them. The files follow the format of 
```python
torch.save({
  'z_vectors': ## An array of numpy.arrays with shape (1, style_dim).
    ## style_dim is typically 512. (aliasfreeganinstance.generator.style_dim)
    ## Values in these numpy.arrays range between 0 and 1
  })
```

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'z_vector_load' + get_output_count())
!python scripts/generate_interpolation.py \
  --method load_z_vectors \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --path_to_z_vectors '/content/drive/MyDrive/colab-alias-free-generate/ros-ffhq-test-0/interpolate-1/z_vectors.pt'

### Create .mp4 with ffmpeg

In [ ]:
!ffmpeg -framerate 24 -i {out_dir}/frames/frame-%09d.png -c:v libx264 -pix_fmt yuv420p -crf 23 {out_dir}/load_z.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### Create .gif with imagemagick

In [ ]:
!convert -delay 5 -loop 0 {out_dir}/frames/*.png {out_dir}/load_z.gif

## Rosinality Translate Script

Creates animation with translation on the same latent.

Run the first cell to install latest version of ffmpeg for the video codacs. Required if using `--video True` argument.

In [ ]:
!python scripts/rosinality_generate.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'rosinality_translate' + get_output_count())
!python scripts/rosinality_generate.py \
    --size {model_size} \
    --outdir {out_dir} \
    --n_frame 48 \
    --n_img 1 \
    --frames True \
    {model_path}

### Create .gif with imagemagick

In [ ]:
!convert -delay 5 -loop 0 {out_dir}/frames/*.png {out_dir}/ros-translate.gif